# Lab 5 playground for mapreduce
This note book will help you test your mapreduce algorithm without deployment on the EMR environment.

This note book is modified from https://colab.research.google.com/drive/17YSsYvnxt_4ZlD2MXOXVyPZNkdW-bA2O

In [1]:
!pip install mrjob

If there are no errors above, then MRJob is properly installed in the system and ready to use.  Let's create a simple MapReduce program to test.  This will save the contents of the cell into a file named mr.py so that we can execute it later.

In [32]:
%%file mr.py
from mrjob.job import MRJob
import re

class MRmyjob(MRJob):
  def mapper(self, _, line):

    # Read each line and only get the first 2 things
    parts = line.strip().split('\t')
    if len(parts) == 5:
      bigram = parts[0]
      times = int(parts[2])
      books = int(parts[4])
      yield bigram, (times, books)

  
  def reducer(self, key, list_of_values):

    total_times = 0
    total_books = 0

    for i in list_of_values:
      total_times += i[0]
      total_books += i[1]

    yield key, total_times/total_books  


if __name__ == '__main__':
    MRmyjob.run()

Overwriting mr.py


Now that the code is saved to a file, we can run it.  This will run it locally (not on Hadoop) and process any file you pass in as the first argument (test_set_5.txt, or your own file to be processed, googlebooks-eng-us-all-2gram-20090715-50-subset.txt).  The result will simply be saved to my_answer.txt

In [39]:
!python mr.py test_sets/test_set_4.txt > my_answer.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\mythr\AppData\Local\Temp\mr.mythr.20240412.181111.958707
Running step 1 of 1...
job output is in C:\Users\mythr\AppData\Local\Temp\mr.mythr.20240412.181111.958707\output
Streaming final output from C:\Users\mythr\AppData\Local\Temp\mr.mythr.20240412.181111.958707\output...
Removing temp directory C:\Users\mythr\AppData\Local\Temp\mr.mythr.20240412.181111.958707...


This piece of code allows you to compare your results to the reference results. Simply upload the solution file to the file directory, say answer_sheet_5.txt

In [40]:
my_answer_set = set([])
reference_set = set([])

with open("challenge2_testsets_solution/answer_sheet_4.txt","r") as d:
  while True:
    line_a = d.readline()
    if not line_a:
      break
    linesplit_a = line_a.split("\t")
    reference_set.add((linesplit_a[0],linesplit_a[1]))

with open('my_answer.txt', 'r') as f:
  while True:
    line_f = f.readline()
    if not line_f:
      break
    linesplit_f = line_f.split("\t")
    my_answer_set.add((linesplit_f[0],linesplit_f[1]))

my_answer_set == reference_set

True

The following code allows you to create your own test set.

In [ ]:
# Generate a random test set
count = 0
import random
random.seed(110)
with open("ngram_old.txt","r") as f:
  with open("test_set.txt","w+") as d:
    while True:
      line = f.readline()
      if not line:
        break
      if random.randint(0,1):
        count = count+1
        d.write(line)
      if count == 500:
        break
